## Spotify Case Study

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

ModuleNotFoundError: No module named 'plotly'

### Reading csv file

In [ ]:
df=pd.read_csv("spotify_dataset.csv")
df

### Columns Name

In [ ]:
df.columns

In [ ]:
df.iloc[0]

In [ ]:
df.info()

###  Changing Datatype of 'Stream' Column to int

In [ ]:
df['Streams'] = df['Streams'].replace(',','', regex=True)
df['Streams'] = df['Streams'].astype('int32')

### Dividing our dataframe on the basis of ranking and  song data

In [ ]:
ranking_data = df[['Song Name', 'Highest Charting Position', 'Number of Times Charted', 'Week of Highest Charting']].copy()
song_data = df[['Song Name', 'Streams', 'Artist',
       'Artist Followers', 'Song ID', 'Genre', 'Release Date', 'Weeks Charted',
       'Popularity','Duration']].copy()

### Calculating the Start and End Highest Charting of a particular song

In [ ]:
ranking_data[['Start Highest Charting', 'End Highest Charting']] = ranking_data['Week of Highest Charting'].str.split('--', expand = True)
ranking_data['Start Highest Charting'] = pd.to_datetime(ranking_data['Start Highest Charting'])
ranking_data['End Highest Charting'] = pd.to_datetime(ranking_data['End Highest Charting'])


### Calculating the charting score of every song 

In [ ]:
ranking_data['Charting Score'] = (ranking_data['Highest Charting Position'] - 201) * (-1)
ranking_data['Charting Score Sum'] = ranking_data['Charting Score'] * ranking_data['Number of Times Charted']


### Plotting Graph of Top 10 Charting Score Sum

In [ ]:
top10_Charting_Score = ranking_data[['Song Name', 'Charting Score Sum']].sort_values(by = 'Charting Score Sum', ascending = False).iloc[:10]
px.bar(top10_Charting_Score, x = 'Song Name', y = 'Charting Score Sum', color = 'Song Name', title = 'Top 10 Charting Score Sum')

### Plotting Graph of Top 10 Charting Score of Ranking 1st

In [ ]:
top10_ranking_1st = ranking_data[ranking_data['Highest Charting Position'] == 1].sort_values(by = 'Number of Times Charted', ascending = False)
top10_ranking_1st = top10_ranking_1st[['Song Name', 'Number of Times Charted']].iloc[:10]
px.bar(top10_ranking_1st, x = 'Song Name', y = 'Number of Times Charted', color = 'Song Name', title = 'Top 10 Charting Score of Ranking 1st')


### Plotting Graph of Top 10 Charting Score of Ranking 2nd

In [ ]:
top10_ranking_2nd = ranking_data[ranking_data['Highest Charting Position'] == 2].sort_values(by = 'Number of Times Charted', ascending = False)
top10_ranking_2nd = top10_ranking_2nd[['Song Name', 'Number of Times Charted']].iloc[:10]
px.bar(top10_ranking_2nd, x = 'Song Name', y = 'Number of Times Charted', color = 'Song Name', title = 'Top 10 Charting Score of Ranking 2nd')


### Plotting Graph of Top 10 Number of Times Charted

In [ ]:
Top10_Number_of_Times_Charted = ranking_data.sort_values(by = 'Number of Times Charted', ascending = False)
Top10_Number_of_Times_Charted = Top10_Number_of_Times_Charted[['Song Name', 'Number of Times Charted']].iloc[:10]
px.bar(Top10_Number_of_Times_Charted, x = 'Song Name', y = 'Number of Times Charted', color = 'Song Name', title = 'Top 10 Number of Times Charted')


### Plotting Graph of Top 10 Most Streamed Song

In [ ]:
sort_streams=df.sort_values(by=['Streams'],ascending=False)
px.bar(sort_streams.head(10),x='Song Name',y='Streams', title="Top 10 Most Streamed Song",color="Song Name")

### Plotting Graph of Top 10 Streamed Genres

In [ ]:
stream_genre=df.groupby('Genre', as_index = False)['Streams'].sum().sort_values(by="Streams", ascending=False)
stream_genre=stream_genre.drop([394],axis=0)
px.bar(stream_genre.head(10),x='Genre',y='Streams', title="Top 10 Streamed Genres", color="Genre")

### Plotting Graph of Top 10 Artists with Highest Numbers of Times Charted

In [ ]:
#here we plot 10 most popular artist based on their appearence in charts
df_numbercharted=df.groupby('Artist').sum().sort_values('Number of Times Charted', ascending=False)
df_numbercharted=df_numbercharted.reset_index()
df_numbercharted
px.bar(df_numbercharted.head(10),x='Artist',y='Number of Times Charted',color='Artist',title="Top 10 Artists with Highest Numbers of Times Charted")


### Calculating Mean Charting Score Sum of every song 

In [ ]:
Charting_date = ranking_data.groupby('Start Highest Charting', as_index = False)['Charting Score Sum'].mean()
Charting_date.head()

### Plotting Graph of Average Charting Score by date

In [ ]:
px.line(Charting_date, x = 'Start Highest Charting', y = 'Charting Score Sum', title = 'Average Charting Score by date')

### Plotting Graph of Top 10 Artists by Popularity

In [ ]:
song_data['Popularity'] = pd.to_numeric(song_data['Popularity'], errors = 'coerce')
#song_data['Artist Followers'] = pd.to_numeric(song_data['Artist Followers'], errors = 'coerce')
artistsbypop = song_data.groupby("Artist")['Popularity'].sum().sort_values(ascending=False)[:10]
artistsbypop=artistsbypop.reset_index()
px.bar(artistsbypop, x="Artist",y="Popularity", color="Artist", title="Top 10 Artists by Popularity")


### Calculating the First and Last Charted Date of a particular song

In [ ]:
song_data['First Charted'] = [song_data['Weeks Charted'].str.split('\n').iloc[i][-1].split('--')[0] for i in range(song_data.shape[0])]
song_data['Last Charted'] = [song_data['Weeks Charted'].str.split('\n').iloc[i][0].split('--')[0] for i in range(song_data.shape[0])]


### Plotting Graph of Top 10 Artist With Most No. Of Songs

In [ ]:
song_data['First Charted'] = pd.to_datetime(song_data['First Charted'], errors = 'coerce')
song_data['Last Charted'] = pd.to_datetime(song_data['Last Charted'], errors = 'coerce')
song_data['On Chart'] = song_data['Last Charted'] - song_data['First Charted']
song_data['On Chart']=song_data['On Chart'].dt.days
artistsbyonchart = song_data.groupby("Artist")['Song Name'].count().sort_values(ascending=False)[:10]
artistsbyonchart=artistsbyonchart.reset_index()
px.bar(artistsbyonchart, x="Artist",y="Song Name", color="Artist", title="Top 10 Artist With Most No. Of Songs", labels={"Song Name": "No. of Songs"})


### Converting Duration From ms to minutes

In [ ]:
song_data['Duration'] = pd.to_numeric(song_data['Duration'].astype(str), errors = 'coerce')
song_data['Duration']=song_data['Duration']/60000

### Plotting Graph of Top 10 Longest Song

In [ ]:
max_dur=song_data.sort_values(by=['Duration'],ascending=False)
px.bar(max_dur.head(10),x="Song Name",y="Duration",color="Song Name",title="Top 10 Longest Song")

### Plotting Graph of Songs Streamed per week  

In [ ]:
firstchartbystream = song_data.groupby(by=['First Charted'])['Streams'].sum()
firstchartbystream=firstchartbystream.reset_index()
firstchartbystream=firstchartbystream.drop([0], axis=0)
firstchartbystream['First Charted']=firstchartbystream['First Charted'].astype(str)
px.bar(firstchartbystream, x='First Charted',y='Streams',color='First Charted',title='Distribution by week')
